# testing metacal

In [2]:
# This module tests our implementation against ngmix
import numpy as np
import ngmix
import galsim
import autometacal

from numpy.testing import assert_allclose

# Generate some data, any old data will do
def make_data(rng, noise, shear):
    """
    simulate an exponential object with moffat psf
    Parameters
    ----------
    rng: np.random.RandomState
        The random number generator
    noise: float
        Noise for the image
    shear: (g1, g2)
        The shear in each component
    Returns
    -------
    ngmix.Observation
    """

    psf_noise = 1.0e-6

    scale = 0.263
    stamp_size = 45
    psf_fwhm = 0.9
    gal_hlr = 0.5
    dy, dx = rng.uniform(low=-scale/2, high=scale/2, size=2)

    psf = galsim.Moffat(
        beta=2.5, fwhm=psf_fwhm,
    ).shear(
        g1=0.0,
        g2=0.0,
    )

    obj0 = galsim.Exponential(
        half_light_radius=gal_hlr,
    ).shear(
        g1=shear[0],
        g2=shear[1],
    ).shift(
        dx=dx,
        dy=dy,
    )

    obj = galsim.Convolve(psf, obj0)

    psf_im = psf.drawImage(nx=stamp_size, ny=stamp_size, scale=scale).array
    im = obj.drawImage(nx=stamp_size, ny=stamp_size, scale=scale).array

    psf_im += rng.normal(scale=psf_noise, size=psf_im.shape)
    im += rng.normal(scale=noise, size=im.shape)

    cen = (np.array(im.shape))/2.0
    psf_cen = (np.array(psf_im.shape))/2.0

    jacobian = ngmix.DiagonalJacobian(
        row=cen[0], col=cen[1], scale=scale,
    )
    psf_jacobian = ngmix.DiagonalJacobian(
        row=psf_cen[0], col=psf_cen[1], scale=scale,
    )

    wt = im*0 + 1.0/noise**2
    psf_wt = psf_im*0 + 1.0/psf_noise**2

    psf_obs = ngmix.Observation(
        psf_im,
        weight=psf_wt,
        jacobian=psf_jacobian,
    )

    obs = ngmix.Observation(
        im,
        weight=wt,
        jacobian=jacobian,
        psf=psf_obs,
    )

    return obs



In [3]:
args={'seed':31415,
      'ntrial':1000,
      'noise': 1e-5,
      'psf': 'gauss'}
shear_true = [0.0, 0.00]
rng = np.random.RandomState(args['seed'])


""" Tests against ngmix
"""
obs = make_data(rng=rng, noise=args['noise'], shear=shear_true)
# We will measure moments with a fixed gaussian weight function
weight_fwhm = 1.2
fitter = ngmix.gaussmom.GaussMom(fwhm=weight_fwhm)
psf_fitter = ngmix.gaussmom.GaussMom(fwhm=weight_fwhm)
# these "runners" run the measurement code on observations
psf_runner = ngmix.runners.PSFRunner(fitter=psf_fitter)
runner = ngmix.runners.Runner(fitter=fitter)

boot = ngmix.metacal.MetacalBootstrapper(
  runner=runner, psf_runner=psf_runner,
  rng=rng,
  psf=args['psf'],
  step=0.01,
  fixnoise=False
)
# Run metacal
resdict, obsdict = boot.go(obs)

# Run autometacal
im = obs.image.reshape(1,45,45).astype('float32')
psf = obs.psf.image.reshape(1,45,45).astype('float32') 
rpsf = obsdict['noshear'].psf.image.reshape(1,45,45).astype('float32') 

mcal = autometacal.generate_mcal_image(im.repeat(3,0), 
                                       psf.repeat(3,0), 
                                       rpsf.repeat(3,0), 
                                       np.array([[0,0],
                                                 [0.01,0],
                                                 [0,0.01]]).astype('float32'))


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [6]:
assert_allclose(mcal[0], obsdict['noshear'].image, atol=1e-5)
assert_allclose(mcal[1], obsdict['1p'].image, atol=2e-5)
assert_allclose(mcal[2], obsdict['2p'].image, atol=2e-5)

In [7]:
im

array([[[ 2.12652049e-05, -3.17956142e-06, -1.77759011e-06, ...,
         -1.96514939e-05,  7.79824586e-06,  7.71479790e-06],
        [-1.64147423e-05,  8.17717137e-06,  9.00753639e-06, ...,
          4.87767875e-06,  1.36765848e-05, -1.30221615e-05],
        [ 2.72141142e-06,  1.46704597e-05,  2.43854129e-06, ...,
          6.69854808e-06, -5.62765672e-06, -1.74260604e-05],
        ...,
        [-2.87779335e-06, -5.77675291e-06, -9.75662806e-06, ...,
         -1.73997933e-06, -1.65110919e-06, -2.26874199e-05],
        [-1.01461128e-05,  9.19423201e-06, -3.35721620e-06, ...,
          1.15719677e-05,  1.41280852e-05, -1.44461262e-06],
        [-1.15097407e-06,  1.39316708e-05,  4.50049674e-06, ...,
          1.38360965e-05,  9.04265744e-06, -1.41828893e-09]]],
      dtype=float32)